# TF CNN Classifier

To run this notebook on an another benchmark, use

```
papermill tf_cnn_classifier.ipynb tf_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
DATASET = 'human_nontata_promoters'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 10

In [2]:
# Parameters
DATASET = "human_nontata_promoters"


In [3]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

human_nontata_promoters 0 64 10


# Data download

In [4]:
from pathlib import Path
import tensorflow as tf

from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer
from genomic_benchmarks.models.tf import basic_cnn_model_v0 as model

if not is_downloaded('human_nontata_promoters'):
    download_dataset('human_nontata_promoters')

2021-10-22 23:43:34.031507: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-22 23:43:34.031521: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


/home/petr/projects/genomic_benchmarks/src/genomic_benchmarks/loc2seq/with_biopython.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


2021-10-22 23:43:36.843684: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-22 23:43:36.843736: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (michle): /proc/driver/nvidia/version does not exist
2021-10-22 23:43:36.844357: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
info(DATASET)

Dataset `human_nontata_promoters` has 2 classes: negative, positive.

All lenghts of genomic intervals equals 251.

Totally 36131 sequences have been found, 27097 for training and 9034 for testing.


/home/petr/projects/genomic_benchmarks/src/genomic_benchmarks/loc2seq/with_biopython.py:96: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


,train,test
negative,12355,4119
positive,14742,4915


## TF Dataset object

In [6]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / DATASET
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 27097 files belonging to 2 classes.


## Text vectorization

In [7]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
#vectorize_layer.set_vocabulary(vocab=np.asarray(['a', 'c', 't', 'g', 'n']))
vectorize_layer.get_vocabulary()

2021-10-22 23:43:37.693825: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


['', '[UNK]', 'g', 'c', 't', 'a']

In [8]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

In [9]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [10]:
history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10


  1/424 [..............................] - ETA: 5:18 - loss: 0.6814 - binary_accuracy: 0.6250

  5/424 [..............................] - ETA: 5s - loss: 0.6551 - binary_accuracy: 0.5813  

  9/424 [..............................] - ETA: 5s - loss: 0.6182 - binary_accuracy: 0.6354

 13/424 [..............................] - ETA: 5s - loss: 0.5844 - binary_accuracy: 0.6731

...

409/424 [===========================>..] - ETA: 0s - loss: 0.2499 - binary_accuracy: 0.8948

413/424 [============================>.] - ETA: 0s - loss: 0.2502 - binary_accuracy: 0.8947

417/424 [============================>.] - ETA: 0s - loss: 0.2501 - binary_accuracy: 0.8947

421/424 [============================>.] - ETA: 0s - loss: 0.2498 - binary_accuracy: 0.8947

424/424 [==============================] - 6s 15ms/step - loss: 0.2494 - binary_accuracy: 0.8949


## Evaluation on the test set

In [11]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

test_ds =  test_dset.map(vectorize_text)

Found 9034 files belonging to 2 classes.


In [12]:
model.evaluate(test_ds)

  1/142 [..............................] - ETA: 26s - loss: 0.5211 - binary_accuracy: 0.8125

 11/142 [=>............................] - ETA: 0s - loss: 0.3510 - binary_accuracy: 0.8480 

 20/142 [===>..........................] - ETA: 0s - loss: 0.3597 - binary_accuracy: 0.8453

 29/142 [=====>........................] - ETA: 0s - loss: 0.3553 - binary_accuracy: 0.8486

 38/142 [=======>......................] - ETA: 0s - loss: 0.3649 - binary_accuracy: 0.8446

...

120/142 [========================>.....] - ETA: 0s - loss: 0.3507 - binary_accuracy: 0.8503

128/142 [==========================>...] - ETA: 0s - loss: 0.3516 - binary_accuracy: 0.8503

137/142 [===========================>..] - ETA: 0s - loss: 0.3502 - binary_accuracy: 0.8516

142/142 [==============================] - 1s 6ms/step - loss: 0.3486 - binary_accuracy: 0.8522


[0.34862691164016724, 0.8522249460220337]